In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import re

# Get the biggest investment managers
* scraping the website: https://www.advratings.com/top-asset-management-firms containing the list of the top asset mangement firms
* processing the name of the of the company
* storing the list of all the companies in funds_list

In [2]:
funds_list = []

url = 'https://www.advratings.com/top-asset-management-firms'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

for row in soup.findAll('table')[0].tbody.findAll('tr'):
    company = str(row.findAll('td')[1].contents)
    company = re.split(r'<|>', company)
    if(len(company) > 2):
        #exluding any special chars and wite spaces from company names
        company = ''.join(e for e in company[2] if e.isalnum())
        funds_list.append(company.upper())
    else:
        company = re.split(r'([\'|\'])', company[0])
        #exluding any special chars and wite spaces from company names
        company = ''.join(e for e in company[2] if e.isalnum())
        funds_list.append(company.upper())

#deleting the first record (remainder of the header)
funds_list = funds_list[1:]
len(funds_list)


56

# Creating a dictionary of {Year : URL list} 
* Getting path to all 13F-HR filing per quarter 
* Each file corresponds to quater 1, the files are for years 2021 - 2018 
* Choosing only files from **funds_list** - list of the top asset investment managers

In [3]:
all_years_urls = {}
path = 'https://www.sec.gov/Archives/'
companies = []

count1 = 0
count2 = 0

file_2021 = open('/home/ivana/Downloads/Data_Preprocessing/13F_2021.txt', 'r')
file_2020 = open('/home/ivana/Downloads/Data_Preprocessing/13F_2020.txt', 'r')
file_2019 = open('/home/ivana/Downloads/Data_Preprocessing/13F_2019.txt', 'r')
file_2018 = open('/home/ivana/Downloads/Data_Preprocessing/13F_2018.txt', 'r')

files = [file_2021, file_2020, file_2019, file_2018]

for file in files:
    forms_url = []
    for line in file:
        
        #parsing out the company name from the list
        company = re.findall(r'13F-HR\s*\d*([\D+\s\D+]*)\s*\d*', line)
        
        #string processing to get uniform formatting
        company = ''.join(e for e in company)
        company = company.replace(' ', '')
        company = re.sub('\d', '', company)
        company = company.upper()

        
        #finding the investment managers that match the list of the top investment mangers *fund_list*
        for name in funds_list[:10]:
            if (company in name or name in company) and len(company) > 3:
                splitted = line.split()
                forms_url.append(path + splitted[-1])
                
    #adding a key:value pair to a dict. - contains 
    all_years_urls[file.name.split('/')[-1]] = forms_url

#finding out how many 
len(all_years_urls.get('13F_2021.txt'))

8

# Getting a data frame
* To obtain a data frame for a desired year (1st quarter of 2021 - 2018)

In [7]:
#list of all year (keys in all_years_urls)
years = ['13F_2021.txt', '13F_2020.txt', '13F_2019.txt', '13F_2018.txt']

names = ['cik','issuer', 'cusip',  'amount', 'sshPrnamtType', 'value']
data_frame = pd.DataFrame(columns=names)

# getting the data frame for a given year 
for url in all_years_urls.get(years[0]): 
    page = requests.get(url)
    data = page.text
    soup = BeautifulSoup(data, "lxml")

    name = []     # Company name
    cusip = []    # CUSIP identifier
    value = []    # Total value of holdings
    amount = []   # Amount of stocks
    sshPrnamtType = [] # sshPramtType
    cik = []

    stocklist = soup.find_all('infotable')

    for s in stocklist:

        if s.find("ns1:nameofissuer") != None:
            # Company name
            n = s.find("ns1:nameofissuer").string
            name.append(n)
            # CUSIP identifier
            c = s.find("ns1:cusip").string
            cusip.append(c)
            # Total value of holdings
            v = int(s.find("ns1:value").string)
            value.append(v)
             # Amount of stocks
            ssh = int(s.find("ns1:shrsorprnamt").find("ns1:sshprnamt").string)
            amount.append(ssh)
            #cik
            c =  soup.find('cik').string
            cik.append(c)
        else:
            # Company name
            n = s.find("nameofissuer").string
            name.append(n)
            # CUSIP identifier
            c = s.find("cusip").string
            cusip.append(c)
            # Total value of holdings
            v = int(s.find("value").string)
            value.append(v)
             # Amount of stocks
            ssh = int(s.find("shrsorprnamt").find("sshprnamt").string)
            amount.append(ssh)
            #cik
            c =  soup.find('cik').string
            cik.append(c)

    stock_dict = {"cik": cik ,"issuer":name,"cusip":cusip,"amount":amount, 'sshPrnamtType':sshPrnamtType, "value":value}
    stock_dict = pd.DataFrame.from_dict(stock_dict, orient='index')
    stock_dict = stock_dict.transpose()
    data_frame = pd.concat([data_frame, stock_dict])


    


In [8]:
data_frame

,cik,issuer,cusip,amount,sshPrnamtType,value
0,0000102909,AAON INC,000360206,81348,None,5420
1,0000102909,AAON INC,000360206,13800,None,919
2,0000102909,AAON INC,000360206,33660,None,2243
3,0000102909,AAON INC,000360206,4149973,None,276513
4,0000102909,AAON INC,000360206,428,None,29
...,...,...,...,...,...,...
27,0001390777,SEAGEN INC,81181C104,1309,None,229
28,0001390777,SEAGEN INC,81181C104,1,None,0
29,0001390777,VELODYNE LIDAR INC,92259F101,965,None,22
30,0001390777,VELODYNE LIDAR INC,92259F101,11610,None,265


# Compressing the DataFrame
* merging row corresponding to the same CIK & same issuer together
* summing the amout of stocks
* deleting value column

In [18]:
aggregation_functions = {'amount': 'sum', 'cik': 'first', 'cusip' : 'first', 'sshPrnamtType' : 'first'}
data_2021 = data_frame.iloc[:,0:5].groupby(data_frame['issuer']).aggregate(aggregation_functions)
data_2021

,amount,cik,cusip,sshPrnamtType
issuer,,,,
10X GENOMICS INC,6187705,0000102909,88025U109,None
111 INC,21402,0000102909,68247Q102,None
180 LIFE SCIENCES CORP,1179639,0000102909,68236V104,None
1LIFE HEALTHCARE INC,7162736,0000102909,68269G107,None
1ST CONSTITUTION BANCORP,419162,0000102909,31986N102,None
...,...,...,...,...
ZUORA INC,9450553,0000102909,98983V106,None
ZYMEWORKS INC,125130,0000102909,98985W102,None
ZYNERBA PHARMACEUTICALS INC,1035402,0000102909,98986X109,None
